## Multiprocessing

Threading vs Multiprocessing

The threading module uses threads, the multiprocessing module uses processes. The difference is that threads run in the same memory space, while processes have separate memory. This makes it a bit harder to share objects between processes with multiprocessing. Since threads use the same memory, precautions have to be taken or two threads will write to the same memory at the same time. This is what the global interpreter lock is for.

Spawning processes is a bit slower than spawning threads.

From: https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python

In [98]:
from multiprocessing import Lock, Process, Value
import time

In [171]:
def add(total, lock):
    for _ in range(1000):
        time.sleep(0.001)
        lock.acquire()
        total.value += 20
        lock.release()
    
def sub(total, lock):
    for _ in range(100):
        time.sleep(0.001)
        lock.acquire()
        total.value -= 20
        lock.release()

#### The lock makes sure that no other thread can write to this value before releasing it.

In [180]:
total = Value('i', 500) # i for integer

lock = Lock()

In [181]:
add_p = Process(target=add, args=(total,lock))
sub_p = Process(target=sub, args=(total,lock))

In [182]:
add_p.start()
sub_p.start()

add_p.join()
sub_p.join()

In [183]:
total.value

18500